In [1]:
import gizmo_analysis as gizmo
import h5py
import halo_analysis as halo
import matplotlib.gridspec as gridspec
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import scipy.interpolate as interp
import scipy.signal as signal
import utilities as ut
from gc_utils import get_descendants_halt, iteration_name, main_prog_halt, snapshot_name, open_snapshot  # type: ignore


In [2]:
sim = "m12i"
sim_dir = "/Users/z5114326/Documents/simulations/"
fire_dir = sim_dir + sim + "/" + sim + "_res7100"

proc_file = sim_dir + sim + "/" + sim + "_processed.hdf5"
proc_data = h5py.File(proc_file, "r")  # open processed data file

snap_dir = fire_dir + "/snapshot_times.txt"
snap_data = pd.read_table(snap_dir, comment="#", header=None, sep=r"\s+")
snap_data.columns = [
    "index",
    "scale_factor",
    "redshift",
    "time_Gyr",
    "lookback_time_Gyr",
    "time_width_Myr",
]

snap_pub_dir = sim_dir + "/snapshot_times_public.txt"
snap_pub_data = pd.read_table(snap_pub_dir, comment="#", header=None, sep=r"\s+")
snap_pub_data.columns = [
    "index",
    "scale_factor",
    "redshift",
    "time_Gyr",
    "lookback_time_Gyr",
    "time_width_Myr",
]

In [14]:
proc_data["it000"]["snapshots"]["snap600"]["gc_id"][8]

122806601

Get COM, and SIGMA of GC system in relation to its host galaxy (pre accretion) and then the MW (post accretion)

In [3]:
halt = halo.io.IO.read_tree(simulation_directory=fire_dir)
# halt = halo.io.IO.read_tree(simulation_directory=fire_dir, assign_hosts_rotation=True)


# in utilities.simulation.Snapshot():
* reading:  Users/z5114326/Documents/simulations/m12i/m12i_res7100/snapshot_times.txt


# in halo_analysis.halo_io.IO():
* read 17859038 halos from:  Users/z5114326/Documents/simulations/m12i/m12i_res7100/halo/rockstar_dm/catalog_hdf5/tree.hdf5



In [4]:
def get_host_at_snap(grp_val, snap_final, acc_snap, halt):
    if grp_val != 0:
        idx = np.where(halt["tid"] == grp_val)[0][0]
        snap = halt["snapshot"][idx]

        if snap < acc_snap:
            while snap > snap_final:
                idx = halt["progenitor.main.index"][idx]
                snap = halt["snapshot"][idx]

        else:
            idx = 0
            snap = halt["snapshot"][idx]

            while snap > snap_final:
                idx = halt["progenitor.main.index"][idx]
                snap = halt["snapshot"][idx]

    else:
        idx = 0
        snap = halt["snapshot"][idx]

        while snap > snap_final:
            idx = halt["progenitor.main.index"][idx]
            snap = halt["snapshot"][idx]

    host_tid = halt["tid"][idx]
    host_idx = idx

    # print(snap, host_tid)

    return host_tid, host_idx

In [5]:
def get_acc_snap(halo_tid, main_halo_tid, halt):
    tid_main_lst = main_prog_halt(halt, main_halo_tid)
    desc_lst = get_descendants_halt(halo_tid, halt)

    idx_lst = np.array([1 if halt["tid"][idx] in tid_main_lst else 0 for idx in desc_lst])
    idx_acc = np.where(idx_lst == 1)[0][0]

    snap_acc = halt["snapshot"][desc_lst[idx_acc]]

    return snap_acc

In [6]:
def get_pos_and_vel(host_index, part, halt):
    dist = ut.particle.get_distances_wrt_center(
        part,
        species=["dark", "star"],
        center_position=halt["position"][host_index],
        rotation=True,
        total_distance=True,
    )

    sp = "star"
    rmax_ctr = 10

    ctr_indices = np.where(dist[sp] < rmax_ctr)[0]

    m = part[sp]["mass"][ctr_indices]
    pos = part[sp]["position"][ctr_indices]
    vel = part[sp]["velocity"][ctr_indices]
    new_ctr = np.multiply(m, pos.T).sum(axis=1) / np.sum(m)
    new_vctr = np.multiply(m, vel.T).sum(axis=1) / np.sum(m)

    new_rot = ut.particle.get_principal_axes(
        part,
        species_name="star",
        distance_max=rmax_ctr,
        mass_percent=90,
        age_percent=25,
        age_limits=None,
        temperature_limits=None,
        center_positions=new_ctr,
        center_velocities=new_vctr,
        host_index=None,
        part_indicess=None,
        return_single_array=True,
        verbose=None,
    )

    dist_vect = ut.particle.get_distances_wrt_center(
        part,
        species=["dark", "star"],
        center_position=new_ctr,
        rotation=new_rot["rotation"],
    )

    vel_vect = ut.particle.get_velocities_wrt_center(
        part,
        species=["dark", "star"],
        center_position=new_ctr,
        center_velocity=new_vctr,
        rotation=new_rot["rotation"],
    )

    # vel_cyl_vect = ut.particle.get_velocities_wrt_center(
    #     part,
    #     species=["dark", "star"],
    #     center_position=halt["position"][0],
    #     center_velocity=halt["velocity"][0],
    #     rotation=new_rot["rotation"],
    #     coordinate_system="cylindrical",
    # )

    return dist_vect, vel_vect

In [7]:
main_halo_tid = 25236877

it_min = 0
# it_max = 100
it_max = 0

snap_lst = [77]

# for snap in snap_pub_data["index"]:
for snap in snap_lst:
    snap_id = snapshot_name(snap)

    part = gizmo.io.Read.read_snapshots(
        ["dark", "star"],
        "index",
        snap,
        simulation_directory=fire_dir,
    )

    grp_vals = set()

    for it in range(it_min, it_max + 1):
        it_id = iteration_name(it)

        snap_dat = proc_data[it_id]["snapshots"][snap_id]

        grp_lst = np.abs(snap_dat["group_id"][()])
        grp_vals.update(grp_lst)

    grp_vals = np.array(sorted(grp_vals))
    grp_vals = [grp_vals[0]]  #############################

    for grp in grp_vals:
        if grp != 0:
            acc_snap = get_acc_snap(grp, main_halo_tid, halt)
        else:
            acc_snap = -1

        _, host_idx = get_host_at_snap(grp, snap, acc_snap, halt)

        # get data for host at snapshot ######################################

        dist_vect, vel_vect = get_pos_and_vel(host_idx, part, halt)

        #######################################################################

        for it in range(it_min, it_max + 1):
            it_id = iteration_name(it)
            snap_dat = proc_data[it_id]["snapshots"][snap_id]

            grp_mask = np.abs(snap_dat["group_id"][()]) == grp

            gc_ids = snap_dat["gc_id"][()][grp_mask]
            ptypes_bin = snap_dat["ptype"][()][grp_mask]

            ptypes = [ptype.decode("utf-8") for ptype in ptypes_bin]

            # pos_lst = np.zeros(len(gc_ids))
            # vel_lst = np.zeros(len(gc_ids))

            gc_id_star_snap = np.array([gc_id for gc_id, ptype in zip(gc_ids, ptypes) if ptype == "star"])
            gc_id_dark_snap = np.array([gc_id for gc_id, ptype in zip(gc_ids, ptypes) if ptype == "dark"])



# in utilities.simulation.Snapshot():
* reading:  Users/z5114326/Documents/simulations/m12i/m12i_res7100/snapshot_times.txt

  using snapshot index = 77, redshift = 4.478


# in gizmo_analysis.gizmo_io.Read():
* reading header from:  Users/z5114326/Documents/simulations/m12i/m12i_res7100/output/snapdir_077/snapshot_077.0.hdf5
  snapshot contains the following number of particles:
    dark      (id = 1): 70514272 particles
    dark2     (id = 2): 5513331 particles
    gas       (id = 0): 70388512 particles
    star      (id = 4): 125821 particles
    blackhole (id = 5): 0 particles

* reading the following
  species: ['dark', 'star']

* reading particles from:
    snapshot_077.0.hdf5
    snapshot_077.1.hdf5
    snapshot_077.2.hdf5
    snapshot_077.3.hdf5

* reading cosmological parameters from:  Users/z5114326/Documents/simulations/m12i/m12i_res7100/initial_condition/ic_agora_m12i.conf

* checking sanity of particle properties


# in gizmo_analysis.gizmo_track.ParticleCoordinate():
  r

In [8]:
gc_star_mask = np.isin(part["star"]["id"], gc_id_star_snap)
gc_dark_mask = np.isin(part["dark"]["id"], gc_id_dark_snap)

gc_id_star = part["star"]["id"][gc_star_mask]
star_pos_lst = dist_vect["star"][gc_star_mask]
star_vel_lst = vel_vect["star"][gc_star_mask]

gc_id_dark = part["dark"]["id"][gc_dark_mask]
dark_pos_lst = dist_vect["dark"][gc_dark_mask]
dark_vel_lst = vel_vect["dark"][gc_dark_mask]

gc_id_lst = np.concatenate((gc_id_star, gc_id_dark))
pos_lst = np.vstack((star_pos_lst, dark_pos_lst))
vel_lst = np.vstack((star_vel_lst, dark_vel_lst))

snap_gc_mask = np.isin(snap_dat["gc_id"][()], gc_id_lst)
log_mass_lst = snap_dat["mass"][()][snap_gc_mask]
mass_lst = 10**log_mass_lst

com = np.multiply(mass_lst, pos_lst.T).sum(axis=1) / np.sum(mass_lst)

In [9]:
com

array([ 5.15068536,  2.47940531, -3.05034703])

Need to work in absoluet distances only. I.e. sigma 3D and distance from COM. This is due to inconsistent frame rotations.

In [10]:
mask = snap_dat["group_id"][()] == 0

x_lst = snap_dat["x"][()][mask]
y_lst = snap_dat["y"][()][mask]
z_lst = snap_dat["z"][()][mask]

pos_lst = np.array([np.array([x, y, z]) for x, y, z in zip(x_lst, y_lst, z_lst)])

log_mass_lst = snap_dat["mass"][()][mask]
mass_lst = 10**log_mass_lst

com = np.multiply(mass_lst, pos_lst.T).sum(axis=1) / np.sum(mass_lst)

com

array([ 55.45009273, -18.22539043, -41.68932847])